In [1]:
import tensorflow
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.utils import multi_gpu_model
#from keras import tensorflow
 
import os
import cv2
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import time
import random as python_random
import argparse
#from generate_true_mean_std import complete_function

from sklearn.model_selection import train_test_split
import keras_metrics as km 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [14]:
#np.random.seed(123)
#python_random.seed(123)
#tensorflow.set_random_seed(1234)

# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-r", "--rounds", type=int, default=1,
#  help="round number")
#args = vars(ap.parse_args())

# grab the number of GPUs and store it in a convenience variable
#R = args["rounds"]
R = 1

data_dir = "D:\\DS_Datasets\\Updated_DS_AVI\\DS1\\DS1Nmax\\"
img_height , img_width = 128, 128
seq_len = 16

#classes = os.listdir(data_dir)

classes = ["class_18"]

#classes = ["class_00", "class_01", "class_02", "class_03", "class_04", "class_05", "class_06", "class_07",
#"class_08", "class_09", "class_10", "class_11", "class_12", "class_13", "class_14", "class_15", "class_16", 
#           "class_17", "class_18", "class_19", "class_20", "class_21"]

#classes = ["class_00", "class_01", "class_02", "class_03", "class_04", "class_05", "class_06", "class_07",
#"class_08", "class_09", "class_10", "class_11", "class_12", "class_13", "class_14", "class_15", "class_16", 
#           "class_19"]

num_gpus = 1
 
#  Creating frames from videos
 
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            #print(vidObj.get(7))
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list
 
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = sorted(os.listdir(input_dir))
    print(classes_list) 
    for c in classes_list:
        if c in classes:
            print(c)
            files_list = os.listdir(os.path.join(input_dir, c))
            for f in files_list:
                frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
                if len(frames) == (seq_len):
                    X.append(frames)
                    y = [0]*len(classes)
                    y[classes.index(c)] = 1
                    Y.append(y)
        else:
            pass
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [15]:
print(classes)
print(len(classes))

['class_18']
1


In [16]:
X, Y = create_data(data_dir)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.95, shuffle=True, random_state=0)

['class_00', 'class_01', 'class_02', 'class_03', 'class_04', 'class_05', 'class_06', 'class_07', 'class_08', 'class_09', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_20', 'class_21']
class_18
Defected frame


In [17]:
model_dir = "D:\\DS_Datasets\\Updated_DS_AVI\\DS1\\convlstm_model_seqlen16.h5"

In [18]:
parallel_model = load_model(model_dir)

In [19]:
y_pred = parallel_model.predict(X_test, verbose=1)
y_pred_report = np.argmax(y_pred, axis=1)
y_test_report = np.argmax(y_test, axis=1)
matrix = confusion_matrix(y_test_report, y_pred_report)
print(matrix)

280/280 [==============================] - 160s 571ms/step
[[  0  66 214]
 [  0   0   0]
 [  0   0   0]]


In [25]:
target_names = ["class_00", "class_01", "class_02", "class_03",
               "class_04", "class_05", "class_06", "class_07",
               "class_08", "class_09", "class_10", "class_11",
               "class_12", "class_13", "class_14", "class_15",
               "class_16", "class_19"]
heat_fluxes = ["2", "4", "6", "8", "10", "12", "16", "18",
              "20", "22", "24", "26", "28", "30", "32",
              "34", "36", "38"]

true_fluxes = []
predicted_fluxes = []

for i in range(len(y_test_report)):
    class_num = y_test_report[i]
    true_fluxes.append(heat_fluxes[class_num])
    
for i in range(len(y_pred_report)):
    class_num = y_pred_report[i]
    predicted_fluxes.append(heat_fluxes[class_num])

In [41]:
R = 1
fluxoutputfilename = r"fluxoutput_ds1nmax_test.txt"

output_file = open(fluxoutputfilename, "w")
print("Flux output file opened")
output_file.write("true fluxes\n")
for i in range(len(true_fluxes)):
    output_file.write(str(true_fluxes[i]))
    output_file.write("\n")
output_file.write("predicted fluxes\n")
for i in range(len(predicted_fluxes)):
    output_file.write(str(predicted_fluxes[i]))
    output_file.write("\n")
output_file.close()
print("Flux output file closed")

statsoutputfilename = r"statsoutput_ds1nmax_test.txt"

complete_function(fluxoutputfilename, statsoutputfilename)

outputfilename = r"output_ds1nmax_test.txt"

output_file = open(outputfilename, "w")

times = time_callback.times

print("Opened txt file")

output_file.write("loss\n")
for i in range(len(loss)):
    output_file.write(str(loss[i]))
    output_file.write("\n")
output_file.write("\nval_loss\n")
for i in range(len(val_loss)):
    output_file.write(str(val_loss[i]))
    output_file.write("\n")
output_file.write("\naccuracy\n")
for i in range(len(accuracy)):
    output_file.write(str(accuracy[i]))
    output_file.write("\n")
output_file.write("\nval_accuracy\n")
for i in range(len(val_accuracy)):
    output_file.write(str(val_accuracy[i]))
    output_file.write("\n")
times = time_callback.times
output_file.write("\nTotal time\n")
output_file.write(str(np.sum(times)))
output_file.write("\nAverage time\n")
output_file.write(str(np.sum(times)/len(times)))
output_file.close()

print("Closed txt file")

print("Total time: ", np.sum(times))
print("Average time: ", np.sum(times)/len(times))


Flux output file opened
Flux output file closed
Inputfile opened
Inputfile closed
true:  0
pred:  1060
trueflux:  ['2\n', '4\n', '6\n', '8\n', '10\n', '12\n', '16\n', '18\n', '20\n', '22\n', '24\n', '26\n', '28\n', '30\n', '32\n', '34\n', '36\n', '38\n']
predflux_float:  [[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], [4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0], [6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.

In [40]:
def split_fluxoutput(fullinput_filename):
    import numpy as np
    
    # Open the full input:
    inputfilename = fullinput_filename
    inputfile = open(inputfilename, "r")
    print("Inputfile opened")
    if (inputfile.mode == 'r'):
        full_lines = inputfile.readlines()
    inputfile.close()
    print("Inputfile closed")
    
    # Read true and pred values to separate arrays:
    true_array = []
    pred_array = []
    for i in range(len(full_lines)):
        if full_lines[i] == "true fluxes\n":
            print("true: ", i)
            j = i+1
            while full_lines[j] != "predicted fluxes\n":
                true_array.append(full_lines[j])
                j += 1
        if full_lines[i] == "predicted fluxes\n":
            print("pred: ", i)
            j = i
            while full_lines:
                j += 1
                try:
                    pred_array.append(full_lines[j])
                except IndexError:
                    break
    
    # Return the two arrays:
    return true_array, pred_array

def calc_mean_std(trueflux, true_array, pred_array):
    import numpy as np
    print("trueflux: ", trueflux)
    
    # Split predictions by corresponding true flux:
    predflux = [ [] for _ in range(len(trueflux))]
    for k in range(len(trueflux)):
        for i in range(len(true_array)):
            if true_array[i] == trueflux[k]:
                item = pred_array[i]
                if item.endswith('\n'): # Cut string formatting
                    item = item[:-1]
                predflux[k].append(item)
    #print("predflux: ", predflux)
    
    # Convert entries into floats:
    predflux_float = [ [] for _ in range(len(trueflux))]
    for k in range(len(predflux_float)):
        for i in range(len(predflux[k])):
            predflux_float[k].append(float(predflux[k][i]))
    print("predflux_float: ", predflux_float)
    
    # Calculate mean and standard deviation:
    means = []
    std = []
    for i in range(len(trueflux)):
        print(i)
        print(sum(predflux_float[i]))
        print(len(predflux_float[i]))
        means.append(sum(predflux_float[i])/len(predflux_float[i]))
        std.append(np.std(predflux_float[i]))
        
    return means, std

def write_to_output(filename, trueflux, means, std):
    import numpy as np
    outputfilename = filename
    
    # Cut off string formatting
    trueflux_cut = []
    for i in range(len(trueflux)):
        item = trueflux[i]
        if item.endswith('\n'):
            item = item[:-1]
            trueflux_cut.append(item)
            
    # Write to 3 columns: true, mean prediction, standard deviation:
    outputfile = open(outputfilename, "+w")
    print("Opened output")
    if outputfile.mode == '+w':
        print("yes")
        outputfile.write("true_flux")
        outputfile.write("\tmean_prediction")
        outputfile.write("\tstandard_deviation\n")
        for i in range(len(trueflux_cut)-1):
            outputfile.write(trueflux_cut[i])
            outputfile.write("\t")
            outputfile.write(str(means[i]))
            outputfile.write("\t")
            outputfile.write(str(std[i]))
            outputfile.write("\n")
    outputfile.close()
    print("Closed output")
    
def complete_function(inputfile, outputfile):
    import numpy as np
    trueflux = ["2\n", "4\n", "6\n", "8\n", "10\n",
           "12\n", "16\n", "18\n", "20\n", "22\n",
           "24\n", "26\n", "28\n", "30\n", "32\n",
           "34\n", "36\n", "38\n"]
    true_array, pred_array = split_fluxoutput(inputfile)
    means, std = calc_mean_std(trueflux, true_array, pred_array)
    write_to_output(outputfile, trueflux, means, std)